# CS 541-B Assignment 2 -- A * search and Uniform Cost Search on NYC Taxi dataset

#### Name: Akshay Atam
#### Stevens ID: 20016304

## NYC Taxi Data
---
The task is to perform Uniform Cost Search and A∗ search. The dataset contains a list of the taxi trips made in NYC in the January 2015. 

Part 1. [20 pts] Represent the data as a graph. For the sake of simplicity, you can assume that edges are only between locations which had a valid taxi trip between them. 

Part 2. [20 pts] Implement the succAndCost fucntion that returns all the connected nodes from the passed node.
And implement the heuristic fucntion using geopy

#### Note: You can install geopy using the command
     pip install geopy

Part 3. [60 pts] 


*   Implement the Uniform Cost search where you can use the trip distances as the edge costs. The program should input two node ids from the user and output the path as well as the cost between them. 
*   Implement A∗ search using a heuristic. One idea of a heuristic value is to use straight line distance between 2 points. This can be computed using the geopy package. The program should input two node ids from the user and output the path as well as the cost between them. 

In [1]:
# Required libraries
import pandas as pd
from tqdm import tqdm
from geopy import distance
import heapq, collections, re, sys, time, os, random

In [2]:
data_NYC = pd.read_csv("NYC_dataset.csv")
data_NYC.head(1)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,nodeid1,nodeid2
0,-73.9641,40.7614,-73.978,40.7831,2.4,48293,28440


In [3]:
def plot_nodes_edges(node1, node2, cost, graph):
    """ Convert a graph in the format of pandas DF as Node 1, Node2, cost into dictionary. 
        Replace the code between ##### with the correct implementation. 

        Note that your code needs to work for any input, not just for the one example below. 

        Input:
              Node 1, Node 2, Cost(	trip_distance) 
              
              example :- 48293, 28440, 2.40
        Output: 
              return graph (Add Key as Node and value as its relevent node and cost)

              Example :- 
              {48293: {2.4: 28440},
               28440: {2.4: 48293}}
              """

    node_cost = {}
    ##############################################START HERE##############################################
    # Part 1 (20 pt)
  
    if node1 in graph:
        graph[node1][cost] = node2
    else:
        graph[node1] = {cost: node2}
    
    # node 2
    if node2 in graph:
        graph[node2][cost] = node1
    else:
        graph[node2] = {cost: node1}
    
    ##############################################END HERE################################################
    return graph
    

def create_nodes_edges(dataset):
    graph = {}
    for i in tqdm(range(dataset.shape[0])):
        graph = plot_nodes_edges(dataset.nodeid1[i], dataset.nodeid2[i], dataset.trip_distance[i], graph)
        graph = plot_nodes_edges(dataset.nodeid2[i], dataset.nodeid1[i], dataset.trip_distance[i], graph)
    return graph 

    
def extract_longitude_latitude(dataset):
    """ Input : 
              (nodeid1, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its pickup_longitude & pickup_latitude
              (nodeid2, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its dropoff_longitude & dropoff_latitude
        Output : 
              {node : [latitude, longitude]} """
    node_lat_long = {}
    for i in tqdm(range(dataset.shape[0])):
        node_lat_long[dataset.nodeid1[i]] = [dataset.pickup_latitude[i],dataset.pickup_longitude[i]]
        node_lat_long[dataset.nodeid2[i]] = [dataset.dropoff_latitude[i],dataset.dropoff_longitude[i]]
    return  node_lat_long

In [4]:
graph = create_nodes_edges(data_NYC)

100%|███████████████████████████████████████████████████████████████████████| 168152/168152 [00:07<00:00, 23997.84it/s]


In [5]:
# calling create_nodes_edges fucntion
node_lat_long = extract_longitude_latitude(data_NYC)

100%|███████████████████████████████████████████████████████████████████████| 168152/168152 [00:06<00:00, 24685.81it/s]


In [6]:
# Data structure for supporting uniform cost search.
class PriorityQueue:
    def  __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {}  # Map from state to priority

    # Insert |state| into the heap with priority |newPriority| if
    # |state| isn't in the heap or |newPriority| is smaller than the existing
    # priority.
    # Return whether the priority queue was updated.
    def update(self, node, newCost, newHistory):
        oldCost = self.priorities.get(node, None)
        if oldCost == None or newCost < oldCost:
            self.priorities[node] = newCost
            heapq.heappush(self.heap, (newCost, node, newHistory))
            return True
        return False

    # Returns (state with minimum priority, priority)
    # or (None, None) if the priority queue is empty.
    def removeMin(self):
        while len(self.heap) > 0:
            priority, state, history = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE: continue  # Outdated priority, skip
            self.priorities[state] = self.DONE
            return (state, priority, history)
        return (None, None, None) # Nothing left...

In [7]:
class PathFinding(object):
    def __init__(self, graph, node_lat_long, start_position, end_position):
        self.graph = graph
        self.node_lat_long = node_lat_long
        self.start_node = start_position
        self.end_node = end_position
        
    def startState(self):
        return self.start_node
    
    def succAndCost(self, current_node):
        """ 
            In this Funciton we have to return all the nodes that are connected to the current_node. 
            
            Note that your code needs to work for any input, not just for the one example below. 

            Input:
                  curent_node
            output: 
            children: all the nodes that are connected to the current node and it's cost.
            Example: children = [(node1,cost1), (node2, cost2)]
        """
        # Generate children
        children = []
        ##############################################START HERE##############################################
        # Part 2.1 (10 pt)
        for cost, next_node in self.graph[current_node].items():
            children.append((next_node, cost))

        ##############################################END HERE################################################        
        return children
    
    def isEnd(self, current_node):
        return self.end_node == current_node
    
    def heuristic(self, node):
        
        """ 
          In this function you need to return the heuristic value of node. 
          Input: 
              Input will be the node
          Output: 
              Output will the distance between the node and the end node, please use distance.geodesic to compute the actual distance between two nodes
        """
        ##############################################START HERE##############################################
          # Part 2.2 (10 pt)
        node_lat_long = self.node_lat_long
        
        # get the starting node and end node
        start = node_lat_long[node]
        end = node_lat_long[self.end_node]
        
        # calculate geodesic distance (in km)
        return distance.geodesic(start, end).km

        ##############################################END HERE################################################

In [8]:
def printSolution(solution):
    totalCost, history = solution
    print_history = 'Starting Node' 
    for node in history:
        print_history += ' -> ' + str(node)
    print(print_history)
    print('totalCost: {}'.format(totalCost))

In [9]:
def astar(problem):
    frontier = PriorityQueue()
    explored = set([])
    """ 
        In this function you have to write an algorithm to calculate the path with the minimum cost using A* Search 
        by calculating it's heuristic cost. Replace the code between the #### with the correct implementation. 
        Input:
              Input will be an object of the class PathFinding
        Output:
              return cost, history
              (history: Path from start node to the end node
              cost: Cost to travel from start node to end node)

                              OR

              return False 
              (if no path is found between Start node and End node)
    """

    ##############################################START HERE##############################################
    # Part 3 (30 pt)
    start_node = problem.startState()
    frontier.update(start_node, problem.heuristic(start_node), [])
    
    # algorithm similar to example done in class
    # difference is in succAndCost function
    # get the first value in the tuple
    while True:
      # Move the minimum cost node from frontier to explored
        current_node, pastCost, history = frontier.removeMin()
        # check if current_node and pastCost is None
        if current_node == None and pastCost == None:
            return -1, []
        explored.add(current_node)
        if problem.isEnd(current_node):
            # TODO: return the past cost and history
            return (pastCost, history)
        current_node_heuristic = problem.heuristic(current_node)
        for child_node, _ in problem.succAndCost(current_node):
            if child_node in explored: 
                continue
            child_heusitic = problem.heuristic(child_node)
            frontier.update(child_node, pastCost + 1 + child_heusitic - current_node_heuristic, history + [child_node])

In [10]:
def UCS(problem):
    frontier = PriorityQueue()
    explored = set([])
    ##############################################START HERE##############################################
    # Part 3 (30 pt)

    s_start = problem.startState()

    # add s_start to frontier
    frontier.update(s_start, 0, [])

    # repeat until frontier is empty
    while True:
        # remove s with smallest priority p from frontier
        s, p, hist = frontier.removeMin()
        
        # check if value of s and p is None
        # this indicates no path available
        if s == None and p == None:
            # return no history and cost -1
            return -1, []

        # if isEnd, return solution
        if problem.isEnd(s):
            return p, hist

        # add s to explored
        explored.add(s)

        for s_, cost in problem.succAndCost(s):
            # if s' in explored
            if s_ in explored:
                continue
            # update frontier with s' and priority p + cost(s,a)
            frontier.update(s_, p + cost, hist + [(s_, cost)])

    ##############################################END HERE################################################

In [11]:
"""
    Below are some of the test cases that you can try
    Start Node = 129891 | End Node = 7381
    Start Node = 51080 | End Node 79375

"""
start_node = 51080
end_node = 79375
problem = PathFinding(graph, node_lat_long, start_node, end_node)

In [12]:
# Astart Algorithm
# Calling the function
astart_path_cost = astar(problem)

# Checking if returned value have cost and history or not
if astart_path_cost != False:
    printSolution(astart_path_cost)

Starting Node -> 124532 -> 44367 -> 101256 -> 50978 -> 139637 -> 32529 -> 141978 -> 55449 -> 115718 -> 79375
totalCost: 10.000000000000002


In [13]:
# UCS Algorithm
# Calling the function
astart_path_cost = UCS(problem)

# Checking if returned value have cost and history or not
if astart_path_cost != False:
    printSolution(astart_path_cost)

Starting Node -> (124532, 1.18) -> (44367, 2.1) -> (101256, 1.1) -> (50978, 0.99) -> (139637, 0.5) -> (32529, 1.4) -> (141978, 1.1) -> (55449, 1.78) -> (115718, 1.4) -> (79375, 2.1)
totalCost: 13.65


In [14]:
src = [129891, 51080, 129891, 7381]
dest = [10666, 79375, 7381, 10666]

for i in range(len(src)):
    if i == 0:
        print("========================================")
        print("Test Case with path available: ")
        print("========================================")
        print()
    if i == 2:
        print("========================================")
        print("Test Case with no path available: ")
        print("========================================")
        print()
        
    problem = PathFinding(graph, node_lat_long, src[i], dest[i])
    print(f"UCS with Start = {src[i]} and End = {dest[i]}: ")
    
    # UCS
    ucs_cost = UCS(problem)
    
    if ucs_cost != False:
        printSolution(ucs_cost)
        print()
    
    print(f"A* with Start = {src[i]} and End = {dest[i]}: ")
    
    # A*
    astar_cost = astar(problem)
    
    if astar_cost != False:
        printSolution(astar_cost)
        print()

Test Case with path available: 

UCS with Start = 129891 and End = 10666: 
Starting Node -> (134204, 0.53) -> (102266, 1.24) -> (47620, 1.44) -> (10470, 0.52) -> (140016, 1.3) -> (121478, 2.77) -> (87417, 0.93) -> (96990, 1.12) -> (93345, 0.53) -> (7651, 1.8) -> (46547, 1.0) -> (10666, 2.5)
totalCost: 15.680000000000001

A* with Start = 129891 and End = 10666: 
Starting Node -> 61208 -> 85044 -> 112281 -> 139234 -> 21676 -> 96562 -> 54431 -> 46547 -> 10666
totalCost: 9.000000000000004

UCS with Start = 51080 and End = 79375: 
Starting Node -> (124532, 1.18) -> (44367, 2.1) -> (101256, 1.1) -> (50978, 0.99) -> (139637, 0.5) -> (32529, 1.4) -> (141978, 1.1) -> (55449, 1.78) -> (115718, 1.4) -> (79375, 2.1)
totalCost: 13.65

A* with Start = 51080 and End = 79375: 
Starting Node -> 124532 -> 44367 -> 101256 -> 50978 -> 139637 -> 32529 -> 141978 -> 55449 -> 115718 -> 79375
totalCost: 10.000000000000002

Test Case with no path available: 

UCS with Start = 129891 and End = 7381: 
Starting No